In [ ]:
!pip install -q pathway bokeh panel

In [ ]:
import pathway as pw
import pandas as pd
import numpy as np
import datetime
import bokeh.plotting
import panel as pn
pn.extension()

In [ ]:
# Download dataset from Google Drive (shared link)
!gdown --fuzzy 'https://drive.google.com/file/d/1Qr1C5puaXsbjfmQYw8qudKtESJPmMg9D/view?usp=sharing' -O dataset.csv

In [ ]:
df = pd.read_csv("dataset.csv")

# Combine date + time → timestamp
df["Timestamp"] = pd.to_datetime(df["LastUpdatedDate"] + " " + df["LastUpdatedTime"],
                                 format="%d-%m-%Y %H:%M:%S")

# Keep only required fields
df = df[["Timestamp", "SystemCodeNumber", "Occupancy", "Capacity"]]
df = df.sort_values("Timestamp")

# Save cleaned data for streaming
df.to_csv("model1_stream.csv", index=False)

In [ ]:
class ParkingSchema(pw.Schema):
    Timestamp: str
    SystemCodeNumber: str
    Occupancy: int
    Capacity: int

# Load stream with 100 rows/second
stream = pw.demo.replay_csv("model1_stream.csv", schema=ParkingSchema, input_rate=100)

# Parse timestamp & day
fmt = "%Y-%m-%d %H:%M:%S"
data = stream.with_columns(
    t = stream.Timestamp.dt.strptime(fmt),
    day = stream.Timestamp.dt.strptime(fmt).dt.strftime("%Y-%m-%dT00:00:00")
)

In [ ]:
windowed = (
    data.windowby(
        time_expr=pw.this.t,
        instance=pw.this.SystemCodeNumber + pw.this.day,
        window=pw.temporal.tumbling(datetime.timedelta(days=1)),
        behavior=pw.temporal.exactly_once_behavior()
    )
    .reduce(
        SystemCodeNumber = pw.reducers.any(pw.this.SystemCodeNumber),
        t = pw.this._pw_window_end,
        occ = pw.reducers.max(pw.this.Occupancy),
        cap = pw.reducers.max(pw.this.Capacity),
    )
    .with_columns(
        price = 10 + 2.5 * (pw.this.occ / pw.this.cap),
        diff = 1
    )
)

In [ ]:
# Get list of lots from CSV
lots = pd.read_csv("model1_stream.csv")["SystemCodeNumber"].unique().tolist()

# Create one Bokeh plot per lot
def plot_linear_price(lot):
    table = windowed.filter(pw.this.SystemCodeNumber == lot).select(
        t = pw.this.t,
        price = pw.this.price
    )
    def make_plot(source):
        fig = bokeh.plotting.figure(
            title=f"Model 1 – Price Curve: {lot}",
            x_axis_type="datetime",
            height=300,
            width=750
        )
        fig.line("t", "price", source=source, color="navy", line_width=2)
        fig.scatter("t", "price", source=source, color="red", size=6)
        return fig
    return table.plot(make_plot, sorting_col="t")

# Stack all plots vertically
all_model1_plots = [plot_linear_price(lot) for lot in lots]
pn.Column("## Model 1 – Daily Linear Pricing Per Lot", *all_model1_plots).servable()

In [ ]:
with contextlib.redirect_stdout(sys.stdout), contextlib.redirect_stderr(sys.stderr):
    pw.run()